# Analyze correlator results
Dec 10, 2020

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
import sys
import glob

In [2]:
%matplotlib widget

In [3]:
from ipywidgets import *

In [4]:
ls /global/cfs/cdirs/m3363/vayyar/cosmogan_data/3ptfnc_stored_results/3dgan_64cube/slice_64/

img_all_gathered_run124-19390_3d_64cube.npy
img_all_gathered_val_3d_64cube.npy
img_idx_0_run124-19390_3d_64cube.npy
img_idx_0_val_3d_64cube.npy
img_idx_10_val_3d_64cube.npy
img_idx_19_run124-19390_3d_64cube.npy
img_idx_1_run124-19390_3d_64cube.npy
img_idx_1_val_3d_64cube.npy
img_idx_20_run124-19390_3d_64cube.npy
img_idx_3_run124-19390_3d_64cube.npy
img_idx_3_val_3d_64cube.npy
img_idx_5_val_3d_64cube.npy
img_idx_6_run124-19390_3d_64cube.npy
img_idx_7_val_3d_64cube.npy
img_idx_8_run124-19390_3d_64cube.npy
img_idx_9_val_3d_64cube.npy


In [5]:
# data_dir='data_stored_results/results_3d_corr6_idx64_1000img/'
# data_dir='data_stored_results/results_3d_corr6_idx32_1000img/'

data_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/3ptfnc_stored_results/3dgan_64cube/slice_32/'


In [7]:
### Gather data into single file
flist=glob.glob(data_dir+'img_idx_*_val*.npy')
print(len(flist))

a1=np.stack([np.load(i) for i in flist])
print(a1.shape)
# np.save(data_dir+'img_all_gathered_val_3d_64cube',a1)

# flist=glob.glob(data_dir+'img_idx_*_run124-19390*.npy')
# print(len(flist))
# a2=np.stack([np.load(i) for i in flist])
# print(a2.shape)
# # np.save(data_dir+'img_all_gathered_run124-19390_3d_64cube',a2)


# flist=glob.glob(data_dir+'img_idx_*_run118-18560*.npy')
# print(len(flist))
# a3=np.stack([np.load(i) for i in flist])
# print(a3.shape)
# # np.save(data_dir+'img_all_gathered_run118-18560_3d_64cube',a3)

193
(193, 8, 8, 8)


In [ ]:
flist=glob.glob(data_dir+'*gathered*.npy')
print(flist)


In [ ]:
cols=['fname','label','data','fle']
df=pd.DataFrame(columns=cols)

for count,fname in enumerate(flist):
    dict1={}
    idx,name=fname.split('/')[-1].split('_')[2],fname.split('/')[-1].split('_')[3]
    label=name+'_'+idx
    print(label)
    val=np.load(fname)
    print(val.shape)
    if len(val.shape)==3: val=np.expand_dims(val,axis=0)
    dict1={'label':label,'data':val,'fle':fname.split('/')[-1],'fname':fname}
    df=df.append(dict1,ignore_index=True)


In [ ]:
# df

In [ ]:
labels_lst=list(df.label.values)
# labels_lst

In [ ]:
df[['label','fle']]

## Plot fixed r1,r2

In [ ]:
def f_get_corrs_fixed_r(arr,r1,r2):
    '''Get correlator values for given r1 and r2 '''    
    num_imgs=arr.shape[0]
    num_corrs=arr.shape[1]
    
    ### Pick r1,r2
    arr=arr[:,:,r1,r2].astype(float)
    # Avg over different images for fixed correlator
    mean=np.array([np.mean(arr[:,i]) for i in range(num_corrs)])
    err=np.array([np.std(arr[:,i])/np.sqrt(num_imgs) for i in range(num_corrs)])
    
    return mean,err

def f_plot_fixed_rs(corrs_arr_lst,labels_lst,r1,r2,mode='relative'):
    ''' mode =['absolute','relative']'''
    
    mean_ref,err_ref=f_get_corrs_fixed_r(corrs_arr_lst[0],r1,r2)
    
    plt.figure()
    for arr,label in zip(corrs_arr_lst,labels_lst):
        mean,err=f_get_corrs_fixed_r(arr,r1,r2)
        if mode=='relative':
            x,y,yerr=np.arange(len(mean)),mean/mean_ref,(mean/mean_ref)*np.sqrt((err/mean)**2 + (err_ref/mean_ref)**2)
        else : 
            x,y,yerr=np.arange(len(mean)),mean,err
        plt.errorbar(x,y,yerr,marker='o',label=label)
    #     plt.plot(x,y,marker='o',label=str(i))
        print(y/yerr)
    plt.legend()
    plt.xlabel('l')
    plt.ylabel('3ptfc')
    plt.title('3ptfc values for fixed {0}-{1} for different l values'.format(r1,r2))

def f_get_corrs_fixed_l(arr,l,r1):
    '''Get correlator values for given l and r1 '''    
    num_imgs=arr.shape[0]
    num_corrs=arr.shape[3]
    print(arr.shape)
    
    ### Pick l,r1
    arr=arr[:,l,r1,:].astype(float)

    # Avg over different images for fixed correlator
    mean=np.array([np.mean(arr[:,i]) for i in range(num_corrs)])
    err=np.array([np.std(arr[:,i])/np.sqrt(num_imgs) for i in range(num_corrs)])
    
    return mean,err


def f_plot_fixed_l(corrs_arr_lst,labels_lst,l,r1,mode='relative'):
    ''' mode =['absolute','relative']'''
    

    mean_ref,err_ref=f_get_corrs_fixed_l(corrs_arr_lst[0],l,r1)

    plt.figure()
    for arr,label in zip(corrs_arr_lst,labels_lst):
        mean,err=f_get_corrs_fixed_l(arr,l,r1)
        
        print(label,type(label))

        if mode=='relative':
            x,y,yerr=np.arange(len(mean)),mean/mean_ref,(mean/mean_ref)*np.sqrt((err/mean)**2 + (err_ref/mean_ref)**2)
        else : 
            x,y,yerr=np.arange(len(mean)),mean,err
        plt.errorbar(x,y,yerr,marker='o',label=label)
    #     plt.plot(x,y,marker='o',label=str(i))
#         print(y/yerr)

    plt.xticks(x)
    plt.xlabel('r2')
    plt.legend()
    plt.title('3ptfc values for fixed l={0} and r1: {1} for different r2 values'.format(l,r1))

# f_plot_fixed_l(corr_lst,['a1','a2'],l=3,r1=3,mode='relative')
# f_plot_fixed_rs(corr_lst,['a1','a2','a3'],r1=1,r2=4,mode='relative')


In [ ]:

def f_plot_df_fixed_l(df,labels_lst,bkgnd_lst,l,r1,mode='relative'):
    ''' mode =['absolute','relative']'''
    
    mean_ref,err_ref=f_get_corrs_fixed_l(df.data.values[0],l,r1)
    
    plt.figure()
    
    df_temp=df[df.label.isin(labels_lst)]
    for i,row in df_temp.iterrows():
        arr=row.data
        label=row.label
        mean,err=f_get_corrs_fixed_l(arr,l,r1)

        if mode=='relative':
            x,y,yerr=np.arange(len(mean)),mean/mean_ref,(mean/mean_ref)*np.sqrt((err/mean)**2 + (err_ref/mean_ref)**2)
        else : 
            x,y,yerr=np.arange(len(mean)),mean,err
        plt.errorbar(x,y,yerr,marker='o',label=label)
    #     plt.plot(x,y,marker='o',label=str(i))
    
    df_temp=df[df.label.isin(bkgnd_lst)]
    for i,row in df_temp.iterrows():
        arr=row.data
        label=row.label
        mean,err=f_get_corrs_fixed_l(arr,l,r1)
        if mode=='relative':
            x,y,yerr=np.arange(len(mean)),mean/mean_ref,(mean/mean_ref)*np.sqrt((err/mean)**2 + (err_ref/mean_ref)**2)
        else : 
            x,y,yerr=np.arange(len(mean)),mean,err
        plt.errorbar(x,y,yerr,label=label,linestyle='--')
       
    plt.xticks(x)
    plt.xlabel('r2')
    plt.ylabel('3ptfc')
    plt.legend()
    plt.title('3ptfc values for fixed l={0} and r1: {1} for different r2 values'.format(l,r1))

def f_plot_df_fixed_rs(df,labels_lst,bkgnd_lst,r1,r2,mode='relative'):
    ''' mode =['absolute','relative']'''
    
    mean_ref,err_ref=f_get_corrs_fixed_r(df.data.values[0],r1,r2)
    
    plt.figure()
    
    df_temp=df[df.label.isin(labels_lst)]
    for i,row in df_temp.iterrows():
        arr=row.data
        label=row.label
        mean,err=f_get_corrs_fixed_r(arr,r1,r2)
        if mode=='relative':
            x,y,yerr=np.arange(len(mean)),mean/mean_ref,(mean/mean_ref)*np.sqrt((err/mean)**2 + (err_ref/mean_ref)**2)
        else : 
            x,y,yerr=np.arange(len(mean)),mean,err
        plt.errorbar(x,y,yerr,marker='o',label=label)
    #     plt.plot(x,y,marker='o',label=str(i))
    
    df_temp=df[df.label.isin(bkgnd_lst)]
    for i,row in df_temp.iterrows():
        arr=row.data
        label=row.label
        mean,err=f_get_corrs_fixed_r(arr,r1,r2)
        if mode=='relative':
            x,y,yerr=np.arange(len(mean)),mean/mean_ref,(mean/mean_ref)*np.sqrt((err/mean)**2 + (err_ref/mean_ref)**2)
        else : 
            x,y,yerr=np.arange(len(mean)),mean,err
        plt.errorbar(x,y,yerr,label=label,linestyle='--')
    
    plt.legend()
    plt.xlabel('l')
    plt.ylabel('3ptfc')
    plt.title('3ptfc values for fixed {0}-{1} for different l values'.format(r1,r2))
    
# f_plot_df_fixed_rs(df,labels_lst[1:],labels_lst,r1=1,r2=3,mode='absolute')
# f_plot_df_fixed_l(df,labels_lst[:1],labels_lst,l=1,r1=1,mode='absolute')

In [ ]:
# df.label.values, labels_lst

In [ ]:
# # labels=['0__0_2d_val_0.5', '3__0_2d_val_0.5','7__0_2d_val_test0.5',
# #         '1__0_2d_val_1.1', '4__0_2d_val_1.1', '8__0_2d_pred1.1_13-31770',
# # #         '2__0_2d_pred0.65_14-34640', '6__0_2d_val_0.65', 
# # #          '5__0_2d_val_0.8',
# #        ]
# labels=df.label.values
# df_temp=df[df.label.isin(labels)]
# img_lst=df_temp.data.values.tolist()
# labels_lst=df_temp.label.values.tolist()
# # labels_lst=['a','b','c','d']
# print(labels_lst)
# f_plot_fixed_l(img_lst,labels_lst,l=3,r1=3,mode='absolute')

In [ ]:
interact(f_plot_df_fixed_rs,df=fixed(df),
                labels_lst=SelectMultiple(options=df.label.values),
                bkgnd_lst=SelectMultiple(options=df.label.values),
                r1=IntSlider(min=0,max=19,step=1),r2=IntSlider(min=0,max=19,step=1),
                mode=['absolute'])


In [ ]:
interact(f_plot_df_fixed_l,df=fixed(df),
                labels_lst=SelectMultiple(options=df.label.values),
                bkgnd_lst=SelectMultiple(options=df.label.values),
                l=ToggleButtons(options=np.arange(7)),
                r1=IntSlider(min=0,max=18,step=1),mode=['absolute'])


In [ ]:
# fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/3ptfnc_stored_results/img_9-corr_2d_val_0.8.npy'
fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/3ptfnc_stored_results/3pt_corr_0_3d_dset1.npy'
fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/3ptfnc_stored_results/img_idx_0_3d_dset1.npy'
a1=np.load(fname)
a1.shape

In [ ]:
# a1.shape

In [ ]:
fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset1_smoothing_const_params_64cube_100k/val.npy'
fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/3ptfnc_stored_results/2d_4sigma-values/3pt_corr_0_1.1.npy'
a1=np.load(fname,mmap_mode='r')


In [ ]:
a1.shape

In [ ]:
a1[0,:].shape